In [ ]:
# @REMOVE-FROM-TEMPLATE
from plaster.tools.ipynb_helpers.displays import restart_kernel; restart_kernel()

In [ ]:
# @IMPORT-MERGE
import os
os.environ["MPLCONFIGDIR"] = "/tmp"
import numpy as np
import pandas as pd
import itertools
import cv2
import random
from scipy.stats import norm, lognorm
from IPython.display import HTML, display
from plaster.tools.log.log import error, debug
from plaster.run.job import JobResult
from plaster.run.run import RunResult
from plaster.run.sigproc_v2 import sigproc_v2_worker as worker
from plaster.run.sigproc_v2.sigproc_v2_result import df_filter, radmat_from_df_filter, df_to_radmat
from plaster.run.sigproc_v2 import fg
from plaster.run.plots import plots, plots_dev
from plaster.run.plots.plots_sigproc import plot_psfs, circle_locs, sigproc_v2_im, sigproc_v2_movie_from_df, sigproc_v2_im_from_df
from plaster.run.plots.plots_sigproc import wizard_xy_df, wizard_scat_df, wizard_raw_images
from plaster.run.sigproc_v2.synth import Synth
from plaster.tools.image.coord import WH, XY, roi_shift, clip2d
from plaster.tools.utils import data
from plaster.tools.zplots import zplots
from plaster.tools.schema import check
from plaster.tools.image import imops
from plaster.tools.zap import zap
from plaster.tools.utils import utils
from plaster.tools.utils import data
from plaster.tools.calibration.calibration import Calibration
from plaster.tools.ipynb_helpers.displays import hd, movie
from plaster.run.base_result import disable_disk_memoize
z = zplots.setup()

In [ ]:
# @REMOVE-FROM-TEMPLATE
from plumbum import local
job = JobResult("/erisyon/internal/jobs_folder/new_abbe7_1t")
run = job.runs[0]

In [ ]:
# Cycle Balancing
ch_i = 0
sig = run.sigproc_v2.sig()[:, ch_i]

one_count_mean = 5000.0  # 5000 is a guess based on abbe7_1t
_sig = sig - one_count_mean
one_count_std = np.percentile(_sig[_sig > 0], 65)  # 65 is a guess based on abbe7_1t

bal = fg.cycle_balance_one_channel(sig, one_count_mean, one_count_std)
corr_sig = sig * bal

with z(_cols=3):
    z.im_sort(sig, _cspan=(0, 20_000), f_title="No cycle balance")
    z.im_sort(corr_sig, _cspan=(0, 20_000), f_title="Cycle balance")
    z.hist(corr_sig, _bins=(-1000, 15000, 1000))


# Estimate zero gain model

In [ ]:
def hist_peak(sig, bins):
    """
    Fit a curve to a historgram to get an estimate of the center of the dominant peak
    using a savgol_filter.
    See https://stackoverflow.com/a/20642478
    """
    from scipy.signal import savgol_filter
    _hist, _edges = np.histogram(sig, bins=bins)
    filt = savgol_filter((_edges[1:], _hist), 101, 3)
    x = filt[0]
    y = filt[1]

    zero_beta = x[np.argmax(y)]
    top = np.max(_hist)
    smooth = y
    
    zero_amp = np.max(smooth)

    # Assume that the samples to the left of this peak are un-contaminated
    # by signal -- so we can use these to get a one-sided std
    zero_sigma = data.one_sided_nanstd(sig)
    
    # Not sure shy but at least on abbe7_1t the zero_sigma estimation by
    # the one-sided method slightly off by about 10%
    zero_sigma *= 1.1

    # Remove the zero-distribution
    y = zero_amp * data.gaussian(bins, zero_beta, zero_sigma)
    return zero_beta, zero_sigma, top, smooth, y[1:], bins[1:]

# Estimate zero gain by examining the histogram near zero
bins = np.linspace(-2000, 8000, 1000)
zero_beta, zero_sigma, top, smooth, zero_curve, x = hist_peak(corr_sig.flatten(), bins)

# PLOT the results
with z(_merge=True):
    z.hist(corr_sig, _bins=bins, f_x_axis_label="inten", f_title=f"field uncorrected")
    z.line(x=x, y=smooth, color="red")
    z.line(x=[zero_beta, zero_beta], y=[0, top], color="red")
    z.line(x=[zero_beta-zero_sigma, zero_beta-zero_sigma], y=[0, top], color="green")
    z.line(x=[zero_beta+zero_sigma, zero_beta+zero_sigma], y=[0, top], color="green")
    z.line(x=x, y=smooth - zero_curve, color="orange" )
    z.line(x=x, y=zero_curve, color="orange" )

# Estimate beta and sigma

In [ ]:
dark = zero_beta + 7 * zero_sigma

# Balance every row by masking out the dark elements
# and removing "one-hit wonders" and "remainders" as those
# are both likely to be contamination
filt_sig = corr_sig.copy()
filt_sig = filt_sig[(filt_sig[:, 1] > dark) & (filt_sig[:, -1] < dark)]
filt_sig[filt_sig < dark] = np.nan
filt_sig = filt_sig[np.any(~np.isnan(filt_sig), axis=1)]

# Filter any row that varies too much -- this is likely scampers, etc
# This make a big difference in abbe5_1t. The lognormal fits way better with this

row_std = np.nanstd(filt_sig, axis=1)
row_std_thresh = np.percentile(row_std, 50)
filt_sig = filt_sig[row_std < row_std_thresh]

# TODO: Add a aspect ratio filter

row_means = np.nanmean(filt_sig, axis=1)
row_means = row_means / np.mean(row_means)
balanced = filt_sig / row_means[:, None]

stack_im = np.hstack((filt_sig, balanced))

balanced = balanced[~np.isnan(balanced)]
log_bal = np.log(balanced)
bins = np.linspace(7, 11, 1000)
beta, sigma = norm.fit(log_bal)
beta = np.exp(beta)

with z(_cols=2):
    z.im_sort(np.nan_to_num(filt_sig))
    model_samples = lognorm.rvs(scale=beta, s=sigma, size=len(balanced))
    with z(_merge=True, _bins=(0, 10_000, 1000), alpha=0.3):
        z.hist(balanced, color="blue")
        z.hist(model_samples, color="red")

print(f"""
    dark      = {dark:>9.3f}
    zero_beta = {zero_beta:>9.3f}
    zero_sigma= {zero_sigma:>9.3f}
    beta      = {beta:>9.3f}
    sigma     = {sigma:>9.3f}
""")